In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import re
# import nltk
# from nltk.book import *
# from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
import lightgbm as lgb
from sklearn.preprocessing import label_binarize
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm_notebook
import joblib
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import defaultdict
# from gensim import corpora
# from gensim import models
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("../1-数据清洗/data_final.csv")
print("data_final: ", data.shape)

data_final:  (73194, 28)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73194 entries, 0 to 73193
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   iyear             73194 non-null  int64  
 1   imonth            73194 non-null  int64  
 2   iday              73194 non-null  int64  
 3   extended          73194 non-null  int64  
 4   country_txt       73194 non-null  object 
 5   region_txt        73194 non-null  object 
 6   provstate         73194 non-null  object 
 7   city              73194 non-null  object 
 8   latitude          73194 non-null  float64
 9   longitude         73194 non-null  float64
 10  specificity       73194 non-null  float64
 11  vicinity          73194 non-null  int64  
 12  summary           73194 non-null  object 
 13  doubtterr         73194 non-null  int64  
 14  multiple          73194 non-null  float64
 15  success           73194 non-null  float64
 16  suicide           73194 non-null  int64 

In [4]:
data.insert(loc=len(data.columns), column="date", value=0)
data.shape

(73194, 29)

In [5]:
d_row = []
for row in tqdm_notebook(range(data.shape[0])):
    if (data.iday[row] == 0):
        d_row.append(row)
data.drop(labels=d_row, axis=0, inplace=True)
data.reset_index(inplace=True, drop=True)
print(data.shape)
print(len(d_row))

  0%|          | 0/73194 [00:00<?, ?it/s]

(73091, 29)
103


In [6]:
import datetime
for row in tqdm_notebook(range(data.shape[0])):
    data.date[row] = datetime.date(data.iyear[row], data.imonth[row], data.iday[row]).isoweekday()
print(data.shape, data["date"])

  0%|          | 0/73091 [00:00<?, ?it/s]

(73091, 29) 0        4
1        5
2        5
3        6
4        2
        ..
73086    4
73087    4
73088    4
73089    4
73090    4
Name: date, Length: 73091, dtype: int64


In [7]:
data.drop(columns=["iyear", "imonth", "iday"], axis=1, inplace=True)
data.shape

(73091, 26)

In [8]:
data.to_csv('data.csv', index=False, encoding='utf_8_sig')

In [39]:
data = pd.read_csv("data.csv")

In [40]:
#将所有:换成.，并且检查最后一个是否为.，否则添加.
for row in tqdm_notebook(range(data.shape[0])):
    data.summary[row] = data.summary[row].replace(";", '.')
#     data.summary[row] = data.summary[row].replace(",", '.')
    if data.summary[row].endswith(".") == False:
        data.summary[row] = "".join([data.summary[row], "."])

  0%|          | 0/73091 [00:00<?, ?it/s]

In [41]:
#删除关键词所在行
for row in tqdm_notebook(range(data.shape[0])):
    content = []
    c_content = []
    j = 0
    i = 0
    dot_count = data.summary[row].count(".")
    for i in range(dot_count):
        content.append(data.summary[row].split('.')[i]) 
#     print(content)
    for j in range(dot_count):
        if ("$" not in content[j]) and ("casualties" not in content[j]) and ("damaged" not in content[j]) and ("damages" not in content[j]) and ("damage" not in content[j]) and ("killed" not in content[j]) and ("injured" not in content[j]) and ("injuries" not in content[j]):
            c_content.append(content[j])
#         print(c_content)
    data.summary[row] = '.'.join(c_content)
# print(data.summary)
data.shape

  0%|          | 0/73091 [00:00<?, ?it/s]

(73091, 26)

In [42]:
data.to_csv('data_txt_test.csv', index=False, encoding='utf_8_sig')

In [47]:
data = pd.read_csv("data_txt_test.csv")
d_row = []
for row in tqdm_notebook(range(data.shape[0])):
    if (pd.isnull(data.summary[row]) == True):
        d_row.append(row)
data.drop(labels=d_row, axis=0, inplace=True)
data.reset_index(inplace=True, drop=True)

  0%|          | 0/73091 [00:00<?, ?it/s]

In [49]:
import re
separator = ':'
for row in tqdm_notebook(range(data.shape[0])):
     if (":" in data.summary[row]):
            data.summary[row] = data.summary[row][data.summary[row].find(':') :]
            data.summary[row] = re.sub(r'[:]', '', data.summary[row])
# print(data.summary)

  0%|          | 0/72550 [00:00<?, ?it/s]

In [50]:
data.to_csv('data_nor.csv', index=False, encoding='utf_8_sig')

In [51]:
n = m = l = z = 0
for row in tqdm_notebook(range(data.shape[0])):
    if (data.risk[row] == 0):
        n = n + 1
    elif (data.risk[row] == 1):
        m = m + 1
    elif (data.risk[row] == 2):
        l = l + 1
    elif (data.risk[row] == 3):
        z = z + 1
print(n, m, l, z, n + m + l + z)

  0%|          | 0/72550 [00:00<?, ?it/s]

10106 16723 22891 22830 72550
